In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
from scipy.stats import mannwhitneyu, ttest_ind
from tqdm import tqdm
import numpy as np 
import math
import seaborn as sns

demos = ['women', 'black', 'hispanic', 'native american']
%matplotlib inline
# hard coded it because of assignment errors

In [12]:
def startup():
    all_unis = {}
    all_completions = {}
    unis = pd.read_csv('../../data/IPEDS/university_info/hd2021.csv', encoding='cp1252')
    for year in range(2008, 2022):
        all_unis[year] = unis
        all_completions[year] = pd.read_csv('../../data/IPEDS/completions/c%s_a.csv' % year)
    demo_names = ['CTOTALW', 'CAIANT', 'CBKAAT', 'CHISPT']
    return all_unis, all_completions, demo_names

In [13]:
def binarize(data, unis, col):
    series = unis[col]
    unique_vals = set(series.unique())
    #binarized_unis = pd.DataFrame()
    for value in unique_vals:
        data['%s_%s'%(col, value)] = (unis[col] == value).astype(int)

In [31]:
def demo_df(all_unis):
    binarized_unis = pd.DataFrame()
    binarized_unis['UNITID'] = all_unis[2021]['UNITID']
    cols = ['SECTOR', 'INSTSIZE', 'UGOFFER', 'GROFFER', 'HBCU', 'HDEGOFR1', 'MEDICAL', 'LOCALE', 'C21BASIC', 'CBSATYPE', 'C21SZSET']
    for col in cols:
        binarize(binarized_unis, all_unis[2021], col)
    return binarized_unis


In [15]:
all_unis, all_completions, demo_names = startup()

In [16]:
all_unis[2021]

,UNITID,INSTNM,IALIAS,ADDR,CITY,STABBR,ZIP,FIPS,OBEREG,CHFNM,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",...,1,290,-2,1089,Madison County,105,-86.568502,34.783368,109,1
1,100663,University of Alabama at Birmingham,UAB,Administration Bldg Suite 1070,Birmingham,AL,35294-0110,1,5,Ray L. Watts,...,1,142,-2,1073,Jefferson County,107,-86.799345,33.505697,93,1
2,100690,Amridge University,Southern Christian University Regions University,1200 Taylor Rd,Montgomery,AL,36117-3553,1,5,Michael C.Turner,...,1,388,-2,1101,Montgomery County,102,-86.174010,32.362609,127,2
3,100706,University of Alabama in Huntsville,UAH University of Alabama Huntsville,301 Sparkman Dr,Huntsville,AL,35899,1,5,Darren Dawson,...,1,290,-2,1089,Madison County,105,-86.640449,34.724557,93,2
4,100724,Alabama State University,,915 S Jackson Street,Montgomery,AL,36104-0271,1,5,Quinton T. Ross,...,1,388,-2,1101,Montgomery County,107,-86.295677,32.364317,99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6284,497301,Avalon Institute-Las Vegas,,"2650 South Decatur Boulevard Suites 1, 6, 8-10",Las Vegas,NV,89102-0046,32,8,Clint Tryon,...,1,332,-2,32003,Clark County,3201,-115.206409,36.142355,45,2
6285,497310,Medspa Academies-National Institute of Modern ...,,3993 Howard Hughes Parkway Suite 150,Las Vegas,NV,89169-6745,32,8,Carrie Brinton,...,1,332,-2,32003,Clark County,3201,-115.158173,36.117236,29,2
6286,497329,American Institute-Cherry Hill,,2201 Route 38 8th Floor,Cherry Hill,NJ,08002-4309,34,2,Christopher Coutts,...,1,428,-2,34007,Camden County,3401,-75.015417,39.939428,57,2
6287,497338,Glendale Career College-North-West College-Bak...,,3000 Ming Avenue,Bakersfield,CA,93304-5075,6,8,Mitchell Fuerst,...,1,-2,-2,6029,Kern County,623,-119.035082,35.339951,221,2


In [32]:
binarized_unis = demo_df(all_unis)

/var/folders/b5/xk6gdm191jgfc3mj34jg7zbh0000gn/T/ipykernel_87755/3321307802.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['%s_%s'%(col, value)] = (unis[col] == value).astype(int)
/var/folders/b5/xk6gdm191jgfc3mj34jg7zbh0000gn/T/ipykernel_87755/3321307802.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['%s_%s'%(col, value)] = (unis[col] == value).astype(int)
/var/folders/b5/xk6gdm191jgfc3mj34jg7zbh0000gn/T/ipykernel_87755/3321307802.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

In [33]:
binarized_unis

,UNITID,SECTOR_0,SECTOR_1,SECTOR_2,SECTOR_3,SECTOR_4,SECTOR_5,SECTOR_6,SECTOR_7,SECTOR_8,...,C21SZSET_10,C21SZSET_11,C21SZSET_12,C21SZSET_13,C21SZSET_14,C21SZSET_15,C21SZSET_16,C21SZSET_17,C21SZSET_18,C21SZSET_-2
0,100654,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,100663,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,100690,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100706,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,100724,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6284,497301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6285,497310,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6286,497329,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6287,497338,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
